In [ ]:
import datetime
from keras.applications import MobileNet
from keras.layers import Input

input_tensor = Input(shape=(200, 200, 3))

#imports the mobilenet model
base_model=MobileNet(weights='imagenet',include_top=False) 

print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential, Model 

#Adding custom Layers 
custom_model=base_model.output
custom_model=GlobalAveragePooling2D()(custom_model)
#custom_model=Dropout(0.5)(custom_model) 
#custom_model=Dense(1024,activation='relu')(custom_model) 
#custom_model=Dense(512,activation='relu')(custom_model) 
#custom_model=Dense(256,activation='relu')(custom_model) 
prediction=Dense(120,activation='softmax')(custom_model) 


# creating the final model 
model_final = Model(inputs=base_model.input,outputs=prediction)

#set the first 20 layers of the network to be non-trainable
for layer in model_final.layers[:20]:
    layer.trainable=False
for layer in model_final.layers[20:]:
    layer.trainable=True
    
# Do not forget to compile it
model_final.compile(loss='categorical_crossentropy',
                     optimizer='Adam',
                     metrics=['accuracy'])   



print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))

In [ ]:
import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

%matplotlib inline

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os

datagen = ImageDataGenerator(
    rescale = 1./255,
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    horizontal_flip=False,
    validation_split=0.2)

data_path = os.path.abspath(os.path.join(os.getcwd(), "data", "dogs"))

train_data_generator = datagen.flow_from_directory(
    directory=data_path,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset='training')

valid_data_generator = datagen.flow_from_directory(
    directory=data_path,
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset='validation')

with open("dogs_classes.txt", 'w', encoding='utf-8') as classes_file:
    for key in train_data_generator.class_indices:
        classes_file.write(str(train_data_generator.class_indices[key]) + ":" + key + "\n")

print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))

In [ ]:

from keras.callbacks import EarlyStopping

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

history = model_final.fit_generator(train_data_generator,
          epochs=12,
          steps_per_epoch=32, 
          verbose=1,
          callbacks=[plot_losses, early],
          validation_data=valid_data_generator,
          validation_steps=8)

score = model_final.evaluate_generator(valid_data_generator, verbose=1, steps=8)

#plot(history)

print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("lego_mobilenet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("lego_mobilenet.h5")
print("Saved model to disk")

print()
print("!!! Done at {}!!!".format(datetime.datetime.now()))